In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import copy
from scipy.signal import savgol_filter
import copy

import pandas as pd
import seaborn as sns
from glob import glob

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
current_path = os.getcwd()

model_folder = current_path + '/models/linearly-growing-layers/'

model = {}

for i in range(1,100):
    model[i] = keras.models.load_model(model_folder+'model_'+str(i)+'.h5')

In [3]:
data_directory = current_path + '/test_data/'

os.chdir(data_directory)
data_files = glob('*CDF_survey.csv')

output_directory = current_path + '/test_data/linearly-growing-layers'

print(data_files)

try:
    os.mkdir(output_directory)
except OSError:
    pass

['046_CDF_survey.csv', '025_CDF_survey.csv', '048_CDF_survey.csv', '144_CDF_survey.csv', '021_CDF_survey.csv', '034_CDF_survey.csv', '137_CDF_survey.csv', '045_CDF_survey.csv', '154_CDF_survey.csv', '139_CDF_survey.csv', '001_CDF_survey.csv', '102_CDF_survey.csv', '110_CDF_survey.csv', '088_CDF_survey.csv', '013_CDF_survey.csv', '081_CDF_survey.csv', '012_CDF_survey.csv', '000_CDF_survey.csv', '017_CDF_survey.csv', '083_CDF_survey.csv', '004_CDF_survey.csv', '082_CDF_survey.csv', '072_CDF_survey.csv', '085_CDF_survey.csv']


In [4]:
c = np.linspace(1,99,99,dtype=int)
quantile_names = [str(ic) for ic in c]

gs = np.linspace(1,11,11,dtype=int)
g_names = ['g'+str(i) for i in gs]

column_names = ['g']+quantile_names

In [6]:
for df in data_files:
    print(df)
#for df in ['000_CDF_survey.csv']:
    #a = pd.read_csv(data_directory+df,header=None).to_numpy()
    a = pd.read_csv(data_directory+df,header=None).to_numpy()

    concs = np.zeros(shape=(a.shape[0],1))
    concs[:,0] = a[:,0]

    response_labels = np.zeros(shape=(a.shape[0],a.shape[0]))

    for j in range(0,a.shape[0]):
        response_labels[j,:] = a[:,1]

    #this_array = np.concatenate((concs,response_labels,a[:,1:]),axis=1)
    mean_array = np.zeros(shape=(a[:,1].shape[0],1))
    mean_array[:,0] = a[:,1]
    this_array = np.concatenate((mean_array,a[:,3:-1]),axis=1)
    #this_array = np.concatenate((concs,a[:,1:]),axis=1)
    #print(this_array.shape)

    test_data = pd.DataFrame(this_array,index=[i for i in range(a.shape[0])],columns=column_names)
    new_data = copy.deepcopy(test_data)
    test_labels = {}

    for i in range(1,this_array.shape[1]):
        #print(column_names[i])
        if i==2:
            test_labels[column_names[i]] = test_data.pop(column_names[i])
        else:
            test_data.pop(column_names[i])
            
    #model_folder = '/Users/sns9/Research/IMS_project/CytometryData/PredictDispersion/model64/'

    for i in range(1,100):
        #model = keras.models.load_model(model_folder+'model_'+str(i)+'.h5')
        
        print(test_data.shape)
        sys.stdout.flush()
        
        #test_predictions = model[i].predict(test_data).flatten()

        test_predictions = model[i].predict(tf.expand_dims(test_data, axis=-1)).flatten()

        new_data[str(i)] = np.maximum(test_predictions,0.0)
        
        if i>1:
            test_data[str(i)] = test_data[str(i-1)] + np.maximum(test_predictions,0.0)
        else:
            test_data[str(i)] = np.maximum(test_predictions,0.0)
            
            
        print('Layer ',i,' completed.')

        #final_data = np.zeros(shape=test_predictions.shape)

        """if i>0:
            for k in range(0,final_data.shape[0]):
                v1 = test_predictions[k]
                v2 = new_data[str(i-1)][k]
                #print(v1,v2)
                final_data[k] = max(v1,v2)

            new_data[str(i)] = final_data

            test_data[str(i)] = final_data
        else:
            new_data[str(i)] = test_predictions

            test_data[str(i)] = test_predictions
        """

    print(df,' completed.')
    
    #for gn in g_names:
    #    new_data.pop(gn)
        
    for iq in range(1,len(quantile_names)):
        new_data[quantile_names[iq]] += new_data[quantile_names[iq-1]]
        
    new_data.to_csv(output_directory+'/new_'+df,index=None,header=None)

046_CDF_survey.csv
(12, 1)


ValueError: in user code:

    File "/Users/swarnavo/miniforge3/lib/python3.9/site-packages/keras/engine/training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "/Users/swarnavo/miniforge3/lib/python3.9/site-packages/keras/engine/training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/swarnavo/miniforge3/lib/python3.9/site-packages/keras/engine/training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/swarnavo/miniforge3/lib/python3.9/site-packages/keras/engine/training.py", line 1791, in predict_step
        return self(x, training=False)
    File "/Users/swarnavo/miniforge3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/swarnavo/miniforge3/lib/python3.9/site-packages/keras/engine/input_spec.py", line 264, in a

a = pd.read_csv(data_file,header=None).to_numpy()

concs = np.zeros(shape=(a.shape[0],1))
concs[:,0] = a[:,0]

response_labels = np.zeros(shape=(a.shape[0],a.shape[0]))

for j in range(0,a.shape[0]):
    response_labels[j,:] = a[:,1]
    
this_array = np.concatenate((concs,response_labels,a[:,1:]),axis=1)
#print(this_array.shape)

test_data = pd.DataFrame(this_array,index=[i for i in range(a.shape[0])],columns=column_names)
new_data = copy.deepcopy(test_data)
test_labels = {}

for i in range(13,this_array.shape[1]):
    #print(column_names[i])
    if i==2:
        test_labels[column_names[i]] = test_data.pop(column_names[i])
    else:
        test_data.pop(column_names[i])

model_folder = '/Users/sns9/Research/IMS_project/CytometryData/PredictDispersion/model64/'

for i in range(0,101):
    model = keras.models.load_model(model_folder+'model_'+str(i)+'.h5')
    
    test_predictions = model.predict(test_data).flatten()
    
    new_data[str(i)] = test_predictions
    
    test_data[str(i)] = test_predictions
    
    final_data = np.zeros(shape=test_predictions.shape)
    
    """if i>0:
        for k in range(0,final_data.shape[0]):
            v1 = test_predictions[k]
            v2 = new_data[str(i-1)][k]
            #print(v1,v2)
            final_data[k] = max(v1,v2)
            
        new_data[str(i)] = final_data
    
        test_data[str(i)] = final_data
    else:
        new_data[str(i)] = test_predictions
    
        test_data[str(i)] = test_predictions
    """
    
print(df,' completed.')

for gn in g_names:
    new_data.pop(gn)

for iq in range(1,len(quantile_names)):
    #print(quantile_names[iq-1],quantile_names[iq])
    new_data[quantile_names[iq]] += new_data[quantile_names[iq-1]]

new_data.to_csv('new_'+data_file,index=None,header=None)

for i in new_data.index:
    data_tag = new_data.loc[i].to_numpy()[:2]
    data = new_data.loc[i].to_numpy()[2:]
    smoothed_data = savgol_filter(data,5,1)
    #print(data_tag.shape,data.shape,smoothed_data.shape)
    out_array = np.concatenate((data_tag,smoothed_data))
    #print(pd.Series(out_array))
    new_data.loc[i,:] = list(out_array)

#print(new_data)